<a href="https://colab.research.google.com/github/Trannguyenphucthinh222/README/blob/master/Cache_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. LIBRARIES**

In [ ]:
%%capture
!pip install --upgrade google-cloud-bigquery
!pip install swifter

import pandas as pd
import numpy as np
import swifter
import requests
import sqlite3
import json
import glob
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from itertools import product
from datetime import datetime,timedelta
from decimal import Decimal
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth
auth.authenticate_user()

In [ ]:
import gspread
import google.auth
credentials, project_id = google.auth.default()
gc = gspread.authorize(credentials)

# **2. SETUP**

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='PROJECT_ID')

In [ ]:
PROJECT_ID = 'internal-blockchain-indexed'

# **3. FUNC**

In [ ]:
def assign_next_year(current_year):
  # query for cache BALANCE (continous calculation)
  balance_cache_query_v2 = f'''
          CREATE OR REPLACE TABLE `analytics_data.eth_sip_balance` AS

          with eth_price as (
            select DATE(TIMESTAMP_SECONDS(block_timestamp)) as block_date,
                  price
            from `ethereum.eth_price`
          ),

          daily_supply as (
            select DATE(block_date) as block_date,
                  daily_supply
            from `ethereum.daily_supply`
          ),

          xfer_raw as (
            select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                  from_address as address,
                  (-1)*(quantity/1e18) as quantity,
            from `ethereum.all_transfers`
            where TIMESTAMP_SECONDS(txn_ts) >= TIMESTAMP '{current_year}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+1}-01-01'
            and from_address not in (select address from `ethereum.categories`) -- exclude cex,dex,contract addresses, mint (basic case)
            and transfer_type in (1,2,3)
            and contract_address = '0x0000000000000000000000000000000000000000'

            union all

            select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                  to_address as address,
                  quantity/1e18 as quantity
            from `ethereum.all_transfers`
            where TIMESTAMP_SECONDS(txn_ts) >= TIMESTAMP '{current_year}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+1}-01-01'
            and to_address not in (select address from `ethereum.categories`)   -- exclude cex,dex,contract addresses, burn
            and transfer_type in (1,2,3)
            and contract_address = '0x0000000000000000000000000000000000000000'
          ),

          transfer as (
            select x.block_date,
                  x.address,
                  x.quantity,
                  p.price
            from xfer_raw x
            left join eth_price p on p.block_date = x.block_date
          ),

          step1 as (  -- value (include selling movements)
            select block_date,
                  address,
                  cumulative_balance as amount,
                  cumulative_value as value
            from `analytics_data.eth_sip_balance`

            union all

            select block_date,
                  address,
                  sum(quantity) as amount,
                  sum(quantity*price) as value
            from transfer
            group by block_date,address
            order by block_date
          ),
          step2 as (  -- cumulative
            select block_date,
                  address,
                  sum(amount) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_balance,
                  sum(value) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_value
            from step1
            order by block_date
          ),

          ranked_data as (
            select block_date,
                    address,
                    cumulative_balance,
                    cumulative_value,
                    ROW_NUMBER() OVER (PARTITION BY address ORDER BY block_date DESC) AS RowNum
            from step2
          )

          --> result
          SELECT DATE('{current_year}-12-31') as block_date,  --latest date of balance data
                address,
                cumulative_balance,
                cumulative_value
          FROM ranked_data
          WHERE RowNum = 1;
      '''

  # psip
  data_cache_query_v2 = f'''
          INSERT INTO `analytics_data.eth_sip_data`(

          with eth_price as (
            select DATE(TIMESTAMP_SECONDS(block_timestamp)) as block_date,
                  price
            from `ethereum.eth_price`
          ),
          daily_supply as (
            select DATE(block_date) as block_date,
                  daily_supply
            from `ethereum.daily_supply`
          ),
          xfer_raw as (
            select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                  from_address as address,
                  (-1)*(quantity/1e18) as quantity,
            from `ethereum.all_transfers`
            where TIMESTAMP_SECONDS(txn_ts) >= '{current_year}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+1}-01-01'
            and from_address not in (select address from `ethereum.categories`) -- exclude cex,dex,contract addresses, mint (basic case)
            and transfer_type in (1,2,3)
            and contract_address = '0x0000000000000000000000000000000000000000'

            union all

            select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                  to_address as address,
                  quantity/1e18 as quantity
            from `ethereum.all_transfers`
            where TIMESTAMP_SECONDS(txn_ts) >= '{current_year}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+1}-01-01'
            and to_address not in (select address from `ethereum.categories`)   -- exclude cex,dex,contract addresses, burn
            and transfer_type in (1,2,3)
            and contract_address = '0x0000000000000000000000000000000000000000'
          ),
          transfer as (
            select x.block_date,
                  x.address,
                  x.quantity,
                  p.price
            from xfer_raw x
            left join eth_price p on p.block_date = x.block_date
          ),
          step1 as (    -- value (include selling movements)
            select block_date,
                  address,
                  sum(amount) as amount,
                  sum(value) as value
            from (select DATE('{current_year}-01-01') as block_date,  -- insert value of the previous calculation
                            address,
                            cumulative_balance as amount,
                            cumulative_value as value
                      from `analytics_data.eth_sip_balance`

                      union all

                      select block_date,
                            address,
                            sum(quantity) as amount,
                            sum(quantity*price) as value
                      from transfer
                      group by block_date, address
                      order by block_date
                  )
            group by block_date, address
            order by block_date
          ),
          step2 as (  -- cumulative
            select block_date,
                  address,
                  sum(amount) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_balance,
                  sum(value) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_value
            from step1
            order by block_date
          ),

          step3 as (
            select block_date,
                  address,
                  cumulative_balance,
                  case
                    when cumulative_balance > 0 then cumulative_value/cumulative_balance
                    else 0
                  end as avg_acq_price
            from step2
          ),

          missed_date as(
          select address,dates as block_date from
          UNNEST(GENERATE_DATE_ARRAY('{current_year}-01-01','{current_year}-12-31', INTERVAL 1 DAY)) AS dates cross join (select distinct address from step3)
          ),

          joined as (
          select distinct m.address, m.block_date, cumulative_balance, avg_acq_price
          from  missed_date m
          left join step3 s on m.address = s.address
          and  s.block_date = m.block_date
          ),

          filled_data as
          (
            select * except(cumulative_balance, avg_acq_price),
              ifnull(last_value(cumulative_balance ignore nulls) over(partition by address order by block_date),0) as cumulative_balance,
              ifnull(last_value(avg_acq_price ignore nulls) over(partition by address order by block_date),0) as avg_acq_price
            from joined
          ),

          final_data as (
            select f.block_date,
                  f.address,
                  f.cumulative_balance,
                  f.avg_acq_price,
                  p.price as current_price
            from filled_data f
            left join eth_price p on p.block_date = f.block_date
            order by f.block_date
          ),

          master_final as (
            select block_date,
                  sum(cumulative_balance) as coin_in_profit
            from final_data
            where avg_acq_price < current_price
            and cumulative_balance > 0
            group by block_date
          )

          --> result
          select f.block_date,
                coin_in_profit*100/d.daily_supply as psip,
                p.price
          from master_final f
          left join daily_supply d on d.block_date = f.block_date
          left join eth_price p on p.block_date = f.block_date
          order by block_date asc
          );
      '''

  return balance_cache_query_v2, data_cache_query_v2

# **4. QUERIES**

## Initial

In [ ]:
current_year = 2015

In [ ]:
# query for cache BALANCE (continous calculation)
balance_cache_query_init = f'''
        create table `analytics_data.eth_sip_balance` as
        with eth_price as (
          select DATE(TIMESTAMP_SECONDS(block_timestamp)) as block_date,
                price
          from `ethereum.eth_price`
        ),

        daily_supply as (
          select DATE(block_date) as block_date,
                daily_supply
          from `ethereum.daily_supply`
        ),

        xfer_raw as (
          select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                from_address as address,
                (-1)*(quantity/1e18) as quantity,
          from `ethereum.all_transfers`
          where TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year + 1}-01-01'
          and from_address not in (select address from `ethereum.categories`) -- exclude cex,dex,contract addresses, mint (basic case)
          and transfer_type in (1,2,3)
          and contract_address = '0x0000000000000000000000000000000000000000'

          union all

          select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                to_address as address,
                quantity/1e18 as quantity
          from `ethereum.all_transfers`
          where TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year + 1}-01-01'
          and to_address not in (select address from `ethereum.categories`)   -- exclude cex,dex,contract addresses, burn
          and transfer_type in (1,2,3)
          and contract_address = '0x0000000000000000000000000000000000000000'
        ),

        transfer as (
          select x.block_date,
                x.address,
                x.quantity,
                p.price
          from xfer_raw x
          left join eth_price p on p.block_date = x.block_date
        ),
        step1 as (  -- value (include selling movements)
          select block_date,
                address,
                sum(quantity) as amount,
                sum(quantity*price) as value
          from transfer
          group by block_date,address
          order by block_date
        ),
        step2 as (  -- cumulative
          select block_date,
                address,
                sum(amount) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_balance,
                sum(value) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_value
          from step1
          order by block_date
        ),
        ranked_data as (
          select block_date,
                  address,
                  cumulative_balance,
                  cumulative_value,
                  ROW_NUMBER() OVER (PARTITION BY address ORDER BY block_date DESC) AS RowNum
          from step2
        )
        --> result
        select DATE('{current_year}-12-31') as block_date,
              address,
              cumulative_balance,
              cumulative_value
        from ranked_data
        where RowNum = 1;
    '''

# query for cache DATA (PSIP)
data_cache_query_init = f'''
        create table `analytics_data.eth_sip_data` as

        with eth_price as (
          select DATE(TIMESTAMP_SECONDS(block_timestamp)) as block_date,
                price
          from `ethereum.eth_price`
        ),
        daily_supply as (
          select DATE(block_date) as block_date,
                daily_supply
          from `ethereum.daily_supply`
        ),
        xfer_raw as (
          select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                from_address as address,
                (-1)*(quantity/1e18) as quantity,
          from `ethereum.all_transfers`
          where TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year + 1}-01-01'
          and from_address not in (select address from `ethereum.categories`) -- exclude cex,dex,contract addresses, mint (basic case)
          and transfer_type in (1,2,3)
          and contract_address = '0x0000000000000000000000000000000000000000'

          union all

          select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
                to_address as address,
                quantity/1e18 as quantity
          from `ethereum.all_transfers`
          where TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year + 1}-01-01'
          and to_address not in (select address from `ethereum.categories`)   -- exclude cex,dex,contract addresses, burn
          and transfer_type in (1,2,3)
          and contract_address = '0x0000000000000000000000000000000000000000'
        ),
        transfer as (
          select x.block_date,
                x.address,
                x.quantity,
                p.price
          from xfer_raw x
          left join eth_price p on p.block_date = x.block_date
        ),
        step1 as (  -- value (include selling movements)
          select block_date,
                address,
                sum(quantity) as amount,
                sum(quantity*price) as value
          from transfer
          group by block_date,address
          order by block_date
        ),
        step2 as (  -- cumulative
          select block_date,
                address,
                sum(amount) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_balance,
                sum(value) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_value
          from step1
          order by block_date
        ),
        step3 as (
          select block_date,
                address,
                cumulative_balance,
                case
                  when cumulative_balance > 0 then cumulative_value/cumulative_balance
                  else 0
                end as avg_acq_price
          from step2
        ),
        missed_date as(
        select address,dates as block_date from
        UNNEST(GENERATE_DATE_ARRAY('{current_year}-08-07','{current_year}-12-31', INTERVAL 1 DAY)) AS dates cross join (select distinct address from step3)
        ),

        joined as (
        select distinct m.address, m.block_date, cumulative_balance, avg_acq_price
        from  missed_date m
        left join step3 s on m.address = s.address
        and  s.block_date = m.block_date
        ),

        filled_data as
        (select * except(cumulative_balance, avg_acq_price),
          ifnull(last_value(cumulative_balance ignore nulls) over(partition by address order by block_date),0) as cumulative_balance,
          ifnull(last_value(avg_acq_price ignore nulls) over(partition by address order by block_date),0) as avg_acq_price
        from joined
        ),

        final_data as (
          select f.block_date,
                f.address,
                f.cumulative_balance,
                f.avg_acq_price,
                p.price as current_price
          from filled_data f
          left join eth_price p on p.block_date = f.block_date
          order by f.block_date
        ),

        master_final as (
          select block_date,
                sum(cumulative_balance) as coin_in_profit
          from final_data
          where avg_acq_price < current_price
          and cumulative_balance > 0
          group by block_date
        )

        --> result
        select f.block_date,
              coin_in_profit*100/d.daily_supply as psip,
              p.price
        from master_final f
        left join daily_supply d on d.block_date = f.block_date
        left join eth_price p on p.block_date = f.block_date
        order by block_date asc
    '''

## Upcoming

In [ ]:
# # query for cache BALANCE (continous calculation)
# balance_cache_query_v2 = f'''
#         CREATE OR REPLACE TABLE `analytics_data.eth_sip_balance` AS

#         with eth_price as (
#           select DATE(TIMESTAMP_SECONDS(block_timestamp)) as block_date,
#                 price
#           from `ethereum.eth_price`
#         ),

#         daily_supply as (
#           select DATE(block_date) as block_date,
#                 daily_supply
#           from `ethereum.daily_supply`
#         ),

#         xfer_raw as (
#           select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
#                 from_address as address,
#                 (-1)*(quantity/1e18) as quantity,
#           from `ethereum.all_transfers`
#           where TIMESTAMP_SECONDS(txn_ts) >= TIMESTAMP '{current_year+1}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+2}-01-01'
#           and from_address not in (select address from `ethereum.categories`) -- exclude cex,dex,contract addresses, mint (basic case)
#           and transfer_type in (1,2,3)
#           and contract_address = '0x0000000000000000000000000000000000000000'

#           union all

#           select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
#                 to_address as address,
#                 quantity/1e18 as quantity
#           from `ethereum.all_transfers`
#           where TIMESTAMP_SECONDS(txn_ts) >= TIMESTAMP '{current_year+1}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+2}-01-01'
#           and to_address not in (select address from `ethereum.categories`)   -- exclude cex,dex,contract addresses, burn
#           and transfer_type in (1,2,3)
#           and contract_address = '0x0000000000000000000000000000000000000000'
#         ),

#         transfer as (
#           select x.block_date,
#                 x.address,
#                 x.quantity,
#                 p.price
#           from xfer_raw x
#           left join eth_price p on p.block_date = x.block_date
#         ),

#         step1 as (  -- value (include selling movements)
#           select block_date,
#                 address,
#                 cumulative_balance as amount,
#                 cumulative_value as value
#           from `analytics_data.eth_sip_balance`

#           union all

#           select block_date,
#                 address,
#                 sum(quantity) as amount,
#                 sum(quantity*price) as value
#           from transfer
#           group by block_date,address
#           order by block_date
#         ),
#         step2 as (  -- cumulative
#           select block_date,
#                 address,
#                 sum(amount) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_balance,
#                 sum(value) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_value
#           from step1
#           order by block_date
#         ),

#         ranked_data as (
#           select block_date,
#                   address,
#                   cumulative_balance,
#                   cumulative_value,
#                   ROW_NUMBER() OVER (PARTITION BY address ORDER BY block_date DESC) AS RowNum
#           from step2
#         )

#         --> result
#         SELECT DATE('{current_year}-12-31') as block_date,  --latest date of balance data
#               address,
#               cumulative_balance,
#               cumulative_value
#         FROM ranked_data
#         WHERE RowNum = 1;
#     '''

# # psip
# data_cache_query_v2 = f'''
#         INSERT INTO `analytics_data.eth_sip_data`(

#         with eth_price as (
#           select DATE(TIMESTAMP_SECONDS(block_timestamp)) as block_date,
#                 price
#           from `ethereum.eth_price`
#         ),
#         daily_supply as (
#           select DATE(block_date) as block_date,
#                 daily_supply
#           from `ethereum.daily_supply`
#         ),
#         xfer_raw as (
#           select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
#                 from_address as address,
#                 (-1)*(quantity/1e18) as quantity,
#           from `ethereum.all_transfers`
#           where TIMESTAMP_SECONDS(txn_ts) >= '{current_year+1}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+2}-01-01'
#           and from_address not in (select address from `ethereum.categories`) -- exclude cex,dex,contract addresses, mint (basic case)
#           and transfer_type in (1,2,3)
#           and contract_address = '0x0000000000000000000000000000000000000000'

#           union all

#           select DATE(TIMESTAMP_SECONDS(txn_ts)) as block_date,
#                 to_address as address,
#                 quantity/1e18 as quantity
#           from `ethereum.all_transfers`
#           where TIMESTAMP_SECONDS(txn_ts) >= '{current_year+1}-01-01' and TIMESTAMP_SECONDS(txn_ts) < TIMESTAMP '{current_year+2}-01-01'
#           and to_address not in (select address from `ethereum.categories`)   -- exclude cex,dex,contract addresses, burn
#           and transfer_type in (1,2,3)
#           and contract_address = '0x0000000000000000000000000000000000000000'
#         ),
#         transfer as (
#           select x.block_date,
#                 x.address,
#                 x.quantity,
#                 p.price
#           from xfer_raw x
#           left join eth_price p on p.block_date = x.block_date
#         ),
#         step1 as (    -- value (include selling movements)
#           select block_date,
#                  address,
#                  sum(amount) as amount,
#                  sum(value) as value
#           from (select DATE('{current_year+1}-01-01') as block_date,  -- insert value of the previous calculation
#                           address,
#                           cumulative_balance as amount,
#                           cumulative_value as value
#                     from `analytics_data.eth_sip_balance`

#                     union all

#                     select block_date,
#                           address,
#                           sum(quantity) as amount,
#                           sum(quantity*price) as value
#                     from transfer
#                     group by block_date, address
#                     order by block_date
#                 )
#           group by block_date, address
#           order by block_date
#         ),
#         step2 as (  -- cumulative
#           select block_date,
#                 address,
#                 sum(amount) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_balance,
#                 sum(value) OVER (PARTITION BY address ORDER BY block_date ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_value
#           from step1
#           order by block_date
#         ),

#         step3 as (
#           select block_date,
#                 address,
#                 cumulative_balance,
#                 case
#                   when cumulative_balance > 0 then cumulative_value/cumulative_balance
#                   else 0
#                 end as avg_acq_price
#           from step2
#         ),

#         missed_date as(
#         select address,dates as block_date from
#         UNNEST(GENERATE_DATE_ARRAY('{current_year+1}-01-01','{current_year+1}-12-31', INTERVAL 1 DAY)) AS dates cross join (select distinct address from step3)
#         ),

#         joined as (
#         select distinct m.address, m.block_date, cumulative_balance, avg_acq_price
#         from  missed_date m
#         left join step3 s on m.address = s.address
#         and  s.block_date = m.block_date
#         ),

#         filled_data as
#         (
#           select * except(cumulative_balance, avg_acq_price),
#             ifnull(last_value(cumulative_balance ignore nulls) over(partition by address order by block_date),0) as cumulative_balance,
#             ifnull(last_value(avg_acq_price ignore nulls) over(partition by address order by block_date),0) as avg_acq_price
#           from joined
#         ),

#         final_data as (
#           select f.block_date,
#                 f.address,
#                 f.cumulative_balance,
#                 f.avg_acq_price,
#                 p.price as current_price
#           from filled_data f
#           left join eth_price p on p.block_date = f.block_date
#           order by f.block_date
#         ),

#         master_final as (
#           select block_date,
#                 sum(cumulative_balance) as coin_in_profit
#           from final_data
#           where avg_acq_price < current_price
#           and cumulative_balance > 0
#           group by block_date
#         )

#         --> result
#         select f.block_date,
#               coin_in_profit*100/d.daily_supply as psip,
#               p.price
#         from master_final f
#         left join daily_supply d on d.block_date = f.block_date
#         left join eth_price p on p.block_date = f.block_date
#         order by block_date asc
#         );
#     '''

# **5. EXECUTION**

### Initial: end of 2015

In [ ]:
current_year = 2015

In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_init, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_init, project_id=PROJECT_ID)

print(f'start year: {current_year} -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

start year: 2015 -> end year: 2015, Dec 31
--- 14.596205472946167 seconds ---


## Upcoming year

### 2016 - 2017

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2016


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

start year: 2016, Jan -> end year: 2016, Dec 31
--- 95.47132968902588 seconds ---


### 2017 - 2018

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2017


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

start year: 2017, Jan -> end year: 2017, Dec 31
--- 1254.9416501522064 seconds ---


### 2018 - 2019

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2018


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

### 2019 - 2020

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2019


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

### 2020 - 2021

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2020


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

### 2021 - 2022

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2021


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

### 2022 - 2023

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2022


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))

### 2023 - 2024

In [ ]:
current_year+=1
print(current_year)
balance_cache_query_v2, data_cache_query_v2 = assign_next_year(current_year)

2023


In [ ]:
start_time = time.time()

# DATA-VIZ cache
pd.io.gbq.read_gbq(data_cache_query_v2, project_id=PROJECT_ID)

# QUERY cache
pd.io.gbq.read_gbq(balance_cache_query_v2, project_id=PROJECT_ID)

print(f'start year: {current_year}, Jan -> end year: {current_year}, Dec 31')

print("--- %s seconds ---" % (time.time() - start_time))